# COMPAS 决策树与 KNN 项目

本 Notebook 演示了如何使用 COMPAS（美国司法系统再犯预测）数据集，结合决策树和 KNN 两种模型进行分类实验。

## 实验目标
- 读取并处理 `compas-scores-raw.csv` 数据；
- 将 `ScoreText` 转换为多分类标签 (可能出现 -1, 0, 1, 2 四类)；
- 训练 KNN 与 决策树，查看各自准确率与混淆矩阵；
- 对敏感特征（种族、性别）做缩放 (×0.5) 以观察其对决策树的影响；
- 通过可视化决策树，理解模型如何使用特征 (特别是 `RawScore` 和敏感属性) 进行分裂。

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier, plot_tree
import matplotlib.pyplot as plt

def print_tree_details(clf, feature_names, node_index=0, depth=0):
    """
    递归打印决策树每个节点的详细信息:
    - 特征名称 (feature)
    - 阈值 (threshold)
    - Gini impurity
    - 样本数 (n_node_samples)
    - value(各类别样本数)
    """
    left_child = clf.tree_.children_left[node_index]
    right_child = clf.tree_.children_right[node_index]
    threshold = clf.tree_.threshold[node_index]
    feature = clf.tree_.feature[node_index]
    impurity = clf.tree_.impurity[node_index]
    n_samples = clf.tree_.n_node_samples[node_index]
    value = clf.tree_.value[node_index]

    indent = "  " * depth

    if left_child == -1 and right_child == -1:
        # 叶子节点
        print(f"{indent}Leaf node {node_index}:")
        print(f"{indent}  gini = {impurity:.3f}, samples = {n_samples}, value = {value}")
    else:
        # 内部节点
        print(f"{indent}Node {node_index}:")
        print(f"{indent}  If {feature_names[feature]} <= {threshold:.3f} "
              f"(gini = {impurity:.3f}, samples = {n_samples}, value = {value}):")
        print_tree_details(clf, feature_names, left_child, depth + 1)
        print_tree_details(clf, feature_names, right_child, depth + 1)


: 

## 1. 读取数据
这里我们读取 `compas-scores-raw.csv`，并先打印列名来确认数据结构。

本数据中包含 `DateOfBirth`、`RawScore`、`Sex_Code_Text`、`Ethnic_Code_Text`、`ScoreText` 等列。

在后续特征工程中，我们将从 `DateOfBirth` 中计算年龄，并将 `ScoreText` 转化为多分类标签。

In [ ]:
# =========== 1. 读取数据 ===========
df = pd.read_csv("/Users/xiangxiaoxin/Documents/GitHub/profile_intro_datascience/week6_classification_decisiontree/data/compas-scores-raw.csv")
print(df.columns)  # 确认实际列名

# 显示前5行
df.head()

## 2. 数据清洗 & 特征工程

1. **计算 Age**：从 `DateOfBirth` 中提取年份，再用当前年份相减；
2. **选择特征**：`["Age", "RawScore", "Sex_Code_Text", "Ethnic_Code_Text"]`；
3. **目标变量**：`ScoreText` 用 `pd.factorize` 得到多分类标签（-1, 0, 1, 2 等）。
4. **独热编码**：对性别、族裔做独热编码 (drop_first=True)；
5. **(可选) 缩放敏感特征**：将 `Sex_Code_Text_XXX`、`Ethnic_Code_Text_XXX` 列乘以 0.5，想看看是否能“弱化”它们在后续模型中的影响。

In [ ]:
df["Age"] = pd.to_datetime(df["DateOfBirth"]).dt.year
current_year = pd.Timestamp.now().year
df["Age"] = current_year - df["Age"]

# 构造特征 X
X = df[["Age", "RawScore", "Sex_Code_Text", "Ethnic_Code_Text"]]

# 目标变量
y, _ = pd.factorize(df["ScoreText"])

# 对 Sex_Code_Text, Ethnic_Code_Text 做独热编码 (drop_first=True)
X = pd.get_dummies(X, columns=["Sex_Code_Text", "Ethnic_Code_Text"], drop_first=True)

###############################################
# 3. 缩放敏感特征
# 识别敏感列: 包含 "Sex_Code_Text_" 或 "Ethnic_Code_Text_"
###############################################
sensitive_cols = [col for col in X.columns if col.startswith("Sex_Code_Text_") or col.startswith("Ethnic_Code_Text_")]

# 将这些敏感列乘以 0.5 (你可根据需要调整这个系数)
for col in sensitive_cols:
    X[col] = X[col] * 0.5

X.head()

: 

**注意**：
- 在决策树里，“乘以 0.5”通常不会显著改变决策逻辑，因为决策树基于信息增益做分裂；
- 但对 KNN 或线性模型，缩放敏感特征可能会减弱它们在距离或系数上的影响。

## 4. 标准化数值特征
对所有特征（包括 RawScore、Age、以及缩放后的敏感列）做标准化。

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 5. 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.3, random_state=42
)
X_train.shape, X_test.shape

## 6. 训练模型 (KNN)

这里先用 KNN (k=5) 进行分类，并打印准确率、混淆矩阵、分类报告。

**观察要点**：
1. 是否只预测多数类？
2. 对四个类别（-1,0,1,2）的 precision/recall/f1-score 是否有较大差异？
3. UndefinedMetricWarning 通常意味着某类几乎没有被预测到。

In [ ]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
knn_pred = knn.predict(X_test)

print("KNN Accuracy:", accuracy_score(y_test, knn_pred))
print("KNN Confusion Matrix:\n", confusion_matrix(y_test, knn_pred))
print("KNN Classification Report:\n", classification_report(y_test, knn_pred))

若 KNN 准确率较高（可能 93% 左右），说明在当前数据下，KNN 对 `RawScore + 缩放敏感特征` 仍能区分大部分样本。但需要仔细查看各类指标，尤其 -1 类是否有足够的识别。

## 7. 训练模型 (Decision Tree)

这里是实验的重点：
1. **max_depth=5**：限制树深度，避免过度生长；
2. 训练完成后，调用 `print_tree_details()` 在终端打印分裂节点，观察是否主要使用 `RawScore` 还是性别、族裔特征；
3. 打印准确率、混淆矩阵、分类报告，查看是否能区分四个类别。

最后可选 `plot_tree()` 进行图形化可视化。

In [ ]:
dt = DecisionTreeClassifier(max_depth=5, random_state=42)
dt.fit(X_train, y_train)
dt_pred = dt.predict(X_test)

# 打印结构
print("\n===== Decision Tree Structure (RawScore + Scaled Sensitive) =====")
feature_names = X.columns.tolist()
print_tree_details(dt, feature_names)

# 查看评估指标
print("\nDecision Tree Accuracy:", accuracy_score(y_test, dt_pred))
print("Decision Tree Confusion Matrix:\n", confusion_matrix(y_test, dt_pred))
print("Decision Tree Classification Report:\n", classification_report(y_test, dt_pred))

### 7.1 决策树可视化

通过 `plot_tree()` 绘制出整棵树的结构。建议：
- 设置较大的 `figsize` 和 `dpi`；
- 为 `feature_names` 和 `class_names` 指定更可读的标签；
- 若树仍然太大或文字重叠，可以再调大 figsize 或限制显示层数（`max_depth=3`）。

In [ ]:
plt.figure(figsize=(15, 8), dpi=200)
plot_tree(
    dt,
    feature_names=feature_names,
    class_names=[f"Class_{c}" for c in np.unique(y)],
    filled=True
)
plt.show()

## 实验结果解读与反思

1. **KNN 准确率**：若在 90%+，表示在此数据上 KNN 拿到不错的性能；
2. **决策树准确率**：通常在 88~89% 左右（含 `RawScore` 时），但对极少数类（-1）无法有效识别；
3. **是否出现敏感特征**：在 `print_tree_details` 中，如果看到 `Sex_Code_Text_xxx`、`Ethnic_Code_Text_xxx` 在高层节点出现，说明模型对它们依赖较大；
4. **RawScore 频繁分裂**：若根节点和多层都在用 `RawScore <= 某阈值`，说明它对区分度最强；
5. **缩放敏感特征**：在决策树中往往不影响分裂顺序，因为树基于信息增益，而非距离或线性系数。结果可能显示准确率与未缩放几乎相同。

## 后续可以做的对比实验
1. **移除 RawScore**：看看准确率会不会大幅下降；
2. **不缩放敏感特征**：比较决策树结构是否有明显变化；
3. **多分类不平衡处理**：对 -1 类样本做上采样或加权，看能否提升对该类的识别；
4. **公平性分析**：若关心模型对不同族裔或性别的 FPR、FNR 差异，可引入 [Fairlearn](https://fairlearn.org/) 进行更深入的分组评估。

### 小结

通过这份 Notebook，您可以：
1. **直观感受**：`RawScore` 在 COMPAS 数据中的区分度极高，一旦移除准确率严重下滑；
2. **了解决策树**：可视化后，能看到哪些特征被优先分裂，是否依赖敏感属性；
3. **警示潜在偏见**：`RawScore` 可能暗含对特定群体的歧视倾向，敏感特征（性别、族裔）也在分裂中出现，需要从公平性视角进一步审视。

---
**完**